<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/5_rgb_xyz_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-09 18:01:03--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11788 (12K) [text/plain]
Saving to: ‘utils.py.1’

utils.py.1          100%[===================>]  11.51K  --.-KB/s    in 0s      

2022-06-09 18:01:03 (81.3 MB/s) - ‘utils.py.1’ saved [11788/11788]



In [5]:
from google.colab import drive
drive.mount('./drive')

Drive already mounted at ./drive; to attempt to forcibly remount, call drive.mount("./drive", force_remount=True).


In [6]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import ModelCreator, get_train_val_ds, get_class_weight

In [7]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [8]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [9]:
train_ds, _ = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3508 files belonging to 16 classes.


In [10]:
classes, class_weight = get_class_weight()

In [11]:
def combine_rgb_xyz(img_batch):
  img_batch_xyz = tfio.experimental.color.rgb_to_xyz(img_batch)
  return (img_batch + img_batch_xyz) / 2

In [12]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (combine_rgb_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [13]:
rgb_xyz_creator = ModelCreator(HUB_URL, 'RGB_XYZ-model')
rgb_xyz_model = rgb_xyz_creator.make_model(img_size=IMG_SIZE)

Model: "RGB_XYZ-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              23500352  
                                                                 
 dense (Dense)               (None, 600)               1229400   
                                                                 
 dropout (Dropout)           (None, 600)               0         
                                                                 
 dense_1 (Dense)             (None, 16)                9616      
                                                                 
Total params: 24,739,368
Trainable params: 1,239,016
Non-trainable params: 23,500,352
_________________________________________________________________


In [14]:
callbacks = rgb_xyz_creator.get_callbacks()

In [ ]:
history = rgb_xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=25,
                        class_weight=class_weight)

Epoch 1/25
519/519 [==============================] - 191s 344ms/step - loss: 1158.2117 - auc: 0.4725 - accuracy: 0.6395
Epoch 2/25
519/519 [==============================] - 175s 334ms/step - loss: 1703.9325 - auc: 0.5370 - accuracy: 0.6982
Epoch 3/25
519/519 [==============================] - 173s 331ms/step - loss: 1564.8438 - auc: 0.5940 - accuracy: 0.7411
Epoch 4/25
519/519 [==============================] - 174s 333ms/step - loss: 1205.7815 - auc: 0.6347 - accuracy: 0.7706
Epoch 5/25
519/519 [==============================] - 176s 336ms/step - loss: 935.1781 - auc: 0.6603 - accuracy: 0.7874
Epoch 6/25
519/519 [==============================] - 178s 341ms/step - loss: 805.5470 - auc: 0.6739 - accuracy: 0.7978
Epoch 7/25
519/519 [==============================] - 174s 333ms/step - loss: 717.5233 - auc: 0.6864 - accuracy: 0.8056
Epoch 8/25
519/519 [==============================] - 174s 333ms/step - loss: 614.1190 - auc: 0.7106 - accuracy: 0.8218
Epoch 9/25
519/519 [================

In [ ]:
rgb_xyz_model.save('./drive/MyDrive/checkpoints/rgb-xyz-model-25.h5')

# ***Second*** day

In [15]:
rgb_xyz_model = tf.keras.models.load_model('./drive/MyDrive/checkpoints/rgb-xyz-model-25.h5',
                                           custom_objects={'KerasLayer': hub.KerasLayer})

In [16]:
history =rgb_xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=40,
                        initial_epoch=25,
                        class_weight=class_weight)

Epoch 26/40
519/519 [==============================] - 1354s 3s/step - loss: 53.7484 - auc: 0.8693 - accuracy: 0.9128
Epoch 27/40
519/519 [==============================] - 177s 339ms/step - loss: 47.7616 - auc: 0.8758 - accuracy: 0.9171
Epoch 28/40
519/519 [==============================] - 174s 332ms/step - loss: 41.6081 - auc: 0.8822 - accuracy: 0.9207
Epoch 29/40
519/519 [==============================] - 182s 348ms/step - loss: 38.0185 - auc: 0.8882 - accuracy: 0.9240
Epoch 30/40
519/519 [==============================] - 178s 341ms/step - loss: 32.8925 - auc: 0.8942 - accuracy: 0.9267
Epoch 31/40
519/519 [==============================] - 183s 351ms/step - loss: 32.2328 - auc: 0.8976 - accuracy: 0.9291
Epoch 32/40
519/519 [==============================] - 187s 358ms/step - loss: 27.8445 - auc: 0.9033 - accuracy: 0.9324
Epoch 33/40
519/519 [==============================] - 180s 345ms/step - loss: 26.8951 - auc: 0.9085 - accuracy: 0.9338
Epoch 34/40
519/519 [=====================

In [17]:
rgb_xyz_model.save('drive/MyDrive/checkpoints/rgb-xyz-model-40.h5')

In [18]:
history =rgb_xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=50,
                        initial_epoch=40,
                        class_weight=class_weight)

Epoch 41/50
519/519 [==============================] - 174s 332ms/step - loss: 13.4663 - auc: 0.9352 - accuracy: 0.9476
Epoch 42/50
519/519 [==============================] - 170s 326ms/step - loss: 12.9879 - auc: 0.9395 - accuracy: 0.9505
Epoch 43/50
519/519 [==============================] - 177s 339ms/step - loss: 13.9674 - auc: 0.9360 - accuracy: 0.9463
Epoch 44/50
519/519 [==============================] - 174s 332ms/step - loss: 12.2783 - auc: 0.9398 - accuracy: 0.9502
Epoch 45/50
519/519 [==============================] - 172s 329ms/step - loss: 11.2335 - auc: 0.9429 - accuracy: 0.9492
Epoch 46/50
519/519 [==============================] - 172s 329ms/step - loss: 10.3868 - auc: 0.9473 - accuracy: 0.9522
Epoch 47/50
519/519 [==============================] - 178s 340ms/step - loss: 11.0245 - auc: 0.9445 - accuracy: 0.9511
Epoch 48/50
519/519 [==============================] - 171s 328ms/step - loss: 10.3962 - auc: 0.9471 - accuracy: 0.9522
Epoch 49/50
519/519 [===================

In [19]:
rgb_xyz_model.save('drive/MyDrive/checkpoints/rgb-xyz-model-50.h5')

In [ ]:
history =rgb_xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=60,
                        initial_epoch=50,
                        class_weight=class_weight)

Epoch 51/60
533/533 [==============================] - 192s 356ms/step - loss: 8.1753 - auc: 0.9343 - accuracy: 0.9447
Epoch 52/60
533/533 [==============================] - 197s 368ms/step - loss: 7.7764 - auc: 0.9379 - accuracy: 0.9454
Epoch 53/60
533/533 [==============================] - 209s 389ms/step - loss: 8.0568 - auc: 0.9398 - accuracy: 0.9468
Epoch 54/60
533/533 [==============================] - 193s 359ms/step - loss: 7.6623 - auc: 0.9397 - accuracy: 0.9437
Epoch 55/60
533/533 [==============================] - 190s 354ms/step - loss: 7.6085 - auc: 0.9405 - accuracy: 0.9464
Epoch 56/60
533/533 [==============================] - 201s 375ms/step - loss: 7.2338 - auc: 0.9419 - accuracy: 0.9458
Epoch 57/60
533/533 [==============================] - 211s 393ms/step - loss: 6.7196 - auc: 0.9442 - accuracy: 0.9478
Epoch 58/60
533/533 [==============================] - 217s 404ms/step - loss: 6.9903 - auc: 0.9403 - accuracy: 0.9473
Epoch 59/60
533/533 [===========================

In [ ]:
rgb_xyz_model.save('drive/MyDrive/checkpoints/rgb-xyz-model-60.h5')

In [ ]:
history =rgb_xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=65,
                        initial_epoch=60,
                        class_weight=class_weight)

Epoch 61/65
533/533 [==============================] - 194s 361ms/step - loss: 6.4579 - auc: 0.9457 - accuracy: 0.9491
Epoch 62/65
533/533 [==============================] - 191s 356ms/step - loss: 6.5886 - auc: 0.9428 - accuracy: 0.9468
Epoch 63/65
533/533 [==============================] - 191s 355ms/step - loss: 6.7383 - auc: 0.9431 - accuracy: 0.9479
Epoch 64/65
533/533 [==============================] - 187s 348ms/step - loss: 6.4305 - auc: 0.9449 - accuracy: 0.9484
Epoch 64: early stopping


In [ ]:
rgb_xyz_model.save_weights('drive/MyDrive/checkpoints/rgb-xyz-model-61.h5')